In [44]:
from ffcv.loader import Loader, OrderOption
from ffcv.fields.decoders import NDArrayDecoder, FloatDecoder

from ffcv.transforms import ToTensor, RandomHorizontalFlip, ToTorchImage, Convert, RandomResizedCropRGBImageDecoder
import torch as ch

ImportError: cannot import name 'RandomResizedCropRGBImageDecoder' from 'ffcv.transforms' (/project/6067616/soroush1/training_fast_publish_faster/.venv/lib/python3.9/site-packages/ffcv/transforms/__init__.py)

In [45]:
BATCH_SIZE = 32
NUM_WORKERS = 1
IMAGE_PATH = "../data/transformed_lamem_256_train.ffcv"
loader = Loader(IMAGE_PATH,
                batch_size=BATCH_SIZE,
                num_workers=NUM_WORKERS,
                order=OrderOption.RANDOM,
                pipelines={
                  'image': [NDArrayDecoder(), RandomHorizontalFlip(), ToTensor(), Convert(ch.float16), ToTorchImage(channels_last=False)],
                  'label': [FloatDecoder(), ToTensor()]
                })
               # )

In [46]:
img, lbl = next(iter(loader))
img.size(), img.dtype, lbl.dtype

(torch.Size([32, 3, 256, 256]), torch.float16, torch.float64)